In [13]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from typing import List


In [9]:
URL_DATA = ["https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-negli-agriturismi-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-negli-esercizi-alberghieri-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-nei-campeggi-e-villaggi-turistici-in-italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-negli-agriturismi-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-negli-esercizi-alberghieri-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-nei-campeggi-e-villaggi-turistici-in-italia-per-regione.csv"]

In [8]:
# Prendiamo i dati degli arrivi e delle presenze
df_arrivi_agriturismi = pd.read_csv(URL_DATA[0], sep=';') 
df_presenze_agriturismi = pd.read_csv(URL_DATA[3], sep=';')

df_arrivi_alberghi = pd.read_csv(URL_DATA[1], sep=';')
df_presenze_alberghi = pd.read_csv(URL_DATA[4], sep=';')

df_arrivi_campeggi = pd.read_csv(URL_DATA[2], sep=';')
df_presenze_campeggi = pd.read_csv(URL_DATA[5], sep=';')

In [22]:
# Creiamo un unico dataframe per categoria
df_agriturismi = pd.merge(df_arrivi_agriturismi, df_presenze_agriturismi, on=['Regione', 'Anno'], how='inner') 
df_alberghi = pd.merge(df_arrivi_alberghi, df_presenze_alberghi, on=['Regione', 'Anno'], how='inner')
df_campeggi = pd.merge(df_arrivi_campeggi, df_presenze_campeggi, on=['Regione', 'Anno'], how='inner')

In [63]:
class Estrattore:
  def __init__(self, data: DataFrame):
    self.data = data.dropna()

  def get_df(self):
    return self.data

  def get_colonne(self, colonne: List[str]):
    return self.data[colonne]
  
  def get_nomi_regioni(self):
    return self.data["Regione"].unique()
  
  def get_regione_by_nome(self, regioneRic: str):
    return self.data[self.data.Regione==regioneRic]
  
  def get_regione_by_anno(self, annoRic: int):
    return self.data[self.data.Anno==annoRic]


In [67]:
e_agriturismi = Estrattore(df_agriturismi)
e_campeggi = Estrattore(df_campeggi)
e_alberghi = Estrattore(df_alberghi)

list_tipi = {}

regioni = e_agriturismi.get_nomi_regioni()
anni = [2008, 2009, 2010, 2011]
categorie = ["Agriturismi", "Campeggi", "Alberghi"]

for regione in regioni:
    list_tipi[regione] = {
        
    }
    for categoria in categorie:
        list_tipi[categoria] =  {anno: (row['Arrivi'], row['Presenze']) for anno in anni for index, row in e_agriturismi.get_regione_by_anno(anno).iterrows()}


display(list_tipi)


{'Piemonte': {},
 'Agriturismi': {2008: (11620, 48099),
  2009: (13231, 45174),
  2010: (11318, 41449),
  2011: (7456, 32483)},
 'Campeggi': {2008: (11620, 48099),
  2009: (13231, 45174),
  2010: (11318, 41449),
  2011: (7456, 32483)},
 'Alberghi': {2008: (11620, 48099),
  2009: (13231, 45174),
  2010: (11318, 41449),
  2011: (7456, 32483)},
 "Valle d'Aosta": {},
 'Lombardia': {},
 'Trentino-Alto Adige': {},
 'Veneto': {},
 'Friuli-Venezia Giulia': {},
 'Liguria': {},
 'Emilia-Romagna': {},
 'Toscana': {},
 'Umbria': {},
 'Marche': {},
 'Lazio': {},
 'Abruzzo': {},
 'Molise': {},
 'Campania': {},
 'Puglia': {},
 'Basilicata': {},
 'Calabria': {},
 'Sicilia': {},
 'Sardegna': {}}